# Bank Financial Data Simple ETL

> For this project, I assumed the role of data engineer working for an international financial company. For this project, I extracted financial data from a websites & a API. After you collect the data, you extract the data of interest to your company and transform it based on the requirements given to you. Once the transformation is complete you load that data into a database.


- Collect data using APIs
- Collect data using webscraping.  
- Perform transformations.
- Load data to csv files.

In [2]:
import pandas as pd
import html5lib
import requests
import time
from datetime import datetime
from bs4 import BeautifulSoup

# Extract Data from API

#### https://exchangeratesapi.io/

In [3]:
api_url = 'https://api.apilayer.com/exchangerates_data/latest?base=EUR&apikey=DS9yDpqy7PQxaJkjrhGqq7lrmMFg7Tm0'

In [4]:
api_response = requests.get(api_url)
print(api_response.status_code)

200


In [40]:
api_data = api_response.json()
api_data

{'success': True,
 'timestamp': 1682963103,
 'base': 'EUR',
 'date': '2023-05-01',
 'rates': {'AED': 4.027439,
  'AFN': 94.48898,
  'ALL': 110.758952,
  'AMD': 425.597376,
  'ANG': 1.972635,
  'AOA': 558.789637,
  'ARS': 243.852081,
  'AUD': 1.653686,
  'AWG': 1.974119,
  'AZN': 1.862869,
  'BAM': 1.948919,
  'BBD': 2.210017,
  'BDT': 116.189778,
  'BGN': 1.947041,
  'BHD': 0.413518,
  'BIF': 2279.613901,
  'BMD': 1.096733,
  'BND': 1.463473,
  'BOB': 7.563228,
  'BRL': 5.472044,
  'BSD': 1.094526,
  'BTC': 3.8762588e-05,
  'BTN': 89.53932,
  'BWP': 14.496817,
  'BYN': 2.762746,
  'BYR': 21495.963869,
  'BZD': 2.206231,
  'CAD': 1.484565,
  'CDF': 2297.655219,
  'CHF': 0.982452,
  'CLF': 0.032071,
  'CLP': 884.932161,
  'CNY': 7.580725,
  'COP': 5163.692443,
  'CRC': 590.43156,
  'CUC': 1.096733,
  'CUP': 29.063421,
  'CVE': 110.715359,
  'CZK': 23.604817,
  'DJF': 194.911373,
  'DKK': 7.453496,
  'DOP': 59.850328,
  'DZD': 148.782774,
  'EGP': 34.057467,
  'ERN': 16.450993,
  'ETB': 5

In [49]:
test_api = pd.json_normalize(api_data['rates'].values())
test_api['shalonn'] = api_data['rates'].keys()
test_api

,shalonn
0,AED
1,AFN
2,ALL
3,AMD
4,ANG
...,...
165,YER
166,ZAR
167,ZMK
168,ZMW


In [52]:
test_api = pd.DataFrame(test_api)
test_api.head()

,shalonn
0,AED
1,AFN
2,ALL
3,AMD
4,ANG


### Transformation 

> One column was imported as a nested dictonary. There I normalized that column to seperated the key from the value. I then created it into a pandas dataframe, appended to the other dataframe with included the entire dataset.

In [41]:
df_api = pd.DataFrame(api_data)
df_api.head(10)

,success,timestamp,base,date,rates
AED,True,1682963103,EUR,2023-05-01,4.027439
AFN,True,1682963103,EUR,2023-05-01,94.488980
ALL,True,1682963103,EUR,2023-05-01,110.758952
AMD,True,1682963103,EUR,2023-05-01,425.597376
ANG,True,1682963103,EUR,2023-05-01,1.972635
AOA,True,1682963103,EUR,2023-05-01,558.789637
ARS,True,1682963103,EUR,2023-05-01,243.852081
AUD,True,1682963103,EUR,2023-05-01,1.653686
AWG,True,1682963103,EUR,2023-05-01,1.974119
AZN,True,1682963103,EUR,2023-05-01,1.862869


In [ ]:
df_api

In [36]:
df_api.dtypes()

TypeError: 'Series' object is not callable

In [35]:
df_api.columns

Index(['success', 'timestamp', 'base', 'date', 'rates'], dtype='object')

# Extract from Webscraping 

In [21]:
bank_url = 'https://www.relbanks.com/worlds-top-banks/assets'
bank_data_response = requests.get(bank_url)

print(bank_data_response.status_code)

200


In [22]:
bank_tables = pd.read_html(bank_url)

In [23]:
bank_df = pd.DataFrame(bank_tables[0])
bank_df.head()

,0,1,2,3,4
0,Rank,Bank,Country,"Total assets, US$b",Balance sheet
1,1,Industrial & Commercial Bank of China,China,4005.58,12/31/2017
2,2,China Construction Bank Corp,China,3397.13,12/31/2017
3,3,Agricultural Bank of China,China,3232.68,12/31/2017
4,4,Bank of China,China,2989.16,12/31/2017


In [24]:
bank_df.columns = ['Rank', 'Bank', 'Country', 'Total_Assets', 'Balance_Sheet']
bank_df

,Rank,Bank,Country,Total_Assets,Balance_Sheet
0,Rank,Bank,Country,"Total assets, US$b",Balance sheet
1,1,Industrial & Commercial Bank of China,China,4005.58,12/31/2017
2,2,China Construction Bank Corp,China,3397.13,12/31/2017
3,3,Agricultural Bank of China,China,3232.68,12/31/2017
4,4,Bank of China,China,2989.16,12/31/2017
...,...,...,...,...,...
116,116,Qatar National Bank,Qatar,221.180,12/31/2017
117,117,National Bank of Canada,Canada,208.408,12/31/2017
118,118,Suntrust Banks,USA,205.962,12/31/2017
119,119,Korea Development Bank,South Korea,205.145,12/31/2016


In [25]:
bank_df = bank_df.iloc[1:, :]
bank_df.head()

,Rank,Bank,Country,Total_Assets,Balance_Sheet
1,1,Industrial & Commercial Bank of China,China,4005.58,12/31/2017
2,2,China Construction Bank Corp,China,3397.13,12/31/2017
3,3,Agricultural Bank of China,China,3232.68,12/31/2017
4,4,Bank of China,China,2989.16,12/31/2017
5,5,Mitsubishi UFJ Financial Group,Japan,2773.82,12/31/2017


In [26]:
bank_df

,Rank,Bank,Country,Total_Assets,Balance_Sheet
1,1,Industrial & Commercial Bank of China,China,4005.58,12/31/2017
2,2,China Construction Bank Corp,China,3397.13,12/31/2017
3,3,Agricultural Bank of China,China,3232.68,12/31/2017
4,4,Bank of China,China,2989.16,12/31/2017
5,5,Mitsubishi UFJ Financial Group,Japan,2773.82,12/31/2017
...,...,...,...,...,...
116,116,Qatar National Bank,Qatar,221.180,12/31/2017
117,117,National Bank of Canada,Canada,208.408,12/31/2017
118,118,Suntrust Banks,USA,205.962,12/31/2017
119,119,Korea Development Bank,South Korea,205.145,12/31/2016


In [ ]:
# Converting API & Webscraping DataFrames to csv 